Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [Dunkin'](https://www.dunkindonuts.com/en/locations) | [HERE API](https://developer.here.com/).
<hr>

# Dunkin' Isochrone Map

It's been over 2 years since my reddit [Distance to Nearest Dunkin'](https://www.reddit.com/r/dataisbeautiful/comments/aqw7lu/oc_ri_distance_to_nearest_dunkin/) post and i've been looking for a geo related Python project. So I decided to revisit and update it to address some of the issues with my first map that people pointed out.

The biggest issue/feedback people had with my first map was the "as the crow flies" distance useage. Meaning the mile circles ignored any road paths, lack of bridges (Prudence Island) or any other obstacles. Using isolines eliminates this issue as it shows drive time going to the Dunkin' and adheres to roadways.

Next, a lot of people pointed out the lack of points from Dunkin's just over the state boundary line. To resolve this, I made a rough trace around the state and buffered it to grab any Dunkin' within 5 miles of Rhode Island to also grabbed the isolines for them.
<hr>

In [1]:
import json
import time
import requests
import pandas as pd
from pathlib import Path

import flexpolyline
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
def isoline_urls(df, time_range, tranport):
    coded_range = ','.join(str(x) for x in time_range)
    
    urls = {}
    for key, lat, lng in zip(df['key'], df['fields.lat'], df['fields.lng']):
        url = "https://isoline.router.hereapi.com/v8/isolines?" + \
             f"apiKey={api_key}" + \
             f"&range[type]=time" + \
             f"&range[values]={coded_range}" + \
             f"&transportMode={transport_type}" + \
             f"&destination={lat},{lng}"

        urls[key] = url
        
    return urls

def isoline_response(key, url):
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
    
    cols = ['id','store_key', 'location_lng', 'location_lat', 'seconds', 'geometry']
    gdf = gpd.GeoDataFrame(columns=cols, crs="EPSG:4326")
    lat = data['arrival']['place']['originalLocation']['lat']
    lng = data['arrival']['place']['originalLocation']['lng']
    
    for i, poly in enumerate(data['isolines']):
        seconds = (poly['range']['value'])
        polygon = poly['polygons'][0]['outer']
        geom = flexpolyline.decode(polygon)
        reverse_geom = [(xy[1], xy[0]) for xy in geom]
        gdf.loc[i] = [i, key, lng, lat, seconds, reverse_geom]
        
    return gdf

def collect_isolines(df, api_key, time_range, transport_type, sleep):
    # filter for already collected keys
    files = list(Path('./data/raw/').glob('*.geojson'))
    collected_keys = [x.stem for x in files]
    df = df[~df['key'].isin(collected_keys)]
    
    isolines = isoline_urls(df, time_range, transport_type)
    
    frames = []
    for key, url in isolines.items():
        gdf = isoline_response(key, url)
        gdf['geometry'] = gdf['geometry'].apply(Polygon)

        frames.append(gdf)
        gdf.to_file(f"./data/raw/{key}.geojson", driver='GeoJSON')
        time.sleep(sleep)
    
    return pd.concat(frames)

In [3]:
with open('./config.json', 'r') as f:
    config = json.load(f)
    
api_key = config['api_key']

time_range = [150, 300, 600, 900]
transport_type = 'car'

df = pd.read_csv('./data/files/ri_dunkins.csv')
gdf = collect_isolines(df, api_key, time_range, transport_type, 5)
gdf.to_file("./data/clean/combined.geojson", driver='GeoJSON')